# Libraries

In [1]:
import pandas as pd
import numpy  as np
import string 
import re
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
# import nltk
# from nltk.corpus import wordnet
# from nltk.stem import WordNetLemmatizer
# #from nltk.tokenize.treebank import TreebankWordDetokenizer
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
#from nltk.corpus import stopwords

from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
import gensim.downloader as api

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import pickle

from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.utils import resample
print('Done')


2022-01-11 22:22:28.845558: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-11 22:22:28.845576: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Done


# Help functions

## Undersampling

In [2]:
def undersample(df, prop = 2):
    
    # get label 2
    df2 = df[df["label"]==2]
    n2 = df2.shape[0]
    
    # get label 0
    df0= df[df["label"]==0]
    df0 = df0.sample(frac=1)
    
    # sample from label 0
    n0 = round(prop*n2)
    df0_sample = df0.head(n0)
    
    # concatenate
    df = pd.concat([df0_sample,df2], axis = 0)
    df = df.sample(frac=1)
    df.reset_index(inplace = True)
    
    assert(n0==prop*n2)
    return df[["text","score_punctuation", "score_capital_word", "label"]]

## Oversampling

In [3]:
def oversample(df, prop = 0.3 , seed = 123):
    n = df.shape[0]
    n2_new = round(prop*n) 
    
    # get label 2
    df2 = df[df["label"]==2]
    n2 = df2.shape[0]
    n2_resample = n2_new - n2
    if (n2_resample) < 0:
        print(f"WARNING: proportion {prop} already satisfied, DF unchanged")
    
    else : 
        indices = list(range(n2))
    
        # resample from df2
        indices_resample = resample(indices, replace=True, n_samples =n2_resample, random_state=f"{seed}")
        df2_resample = df2.iloc[indices_resample]

        df2_new = pd.concat([df2_resample, df2], axis =0)

        df_new = pd.concat([df2_new, df[df["label"]==0]], axis =0)
        df = df_new
    #assert(df2_new.shape[0]==prop*n)
    
    return df


## Data cleaning and tokenizing function

In [4]:
# # clean each tweet
# def clean_data(data, remove_special_char_2lower_case = True):
    
#     #Removing URLs with a regular expression
#     url_pattern = re.compile(r'https?://\S+|www\.\S+')
#     data = url_pattern.sub(r'', data)

#     # Remove Emails
#     data = re.sub('\S*@\S*\s?', '', data)
   
#     # tokenize + remove scpecial characters + set to lower case
#     if remove_special_char_2lower_case == True:
#         data = text_to_word_sequence(data) 
#     else :
#     # tokenize 
#         data = data.split() 
    
    
#     # Remove stopwords
#     stop_words = set(stopwords.words('english')) 
#     data = [w for w in data if not w in stop_words]         
    
#     # Remove digits
#     data = ' '.join(word for word in data if not word.isdigit())
    
    
#     return text_to_word_sequence(data)

# # clean dataset
# def apply_data_cleaning(X, text, drop_text = False, remove_special_char_2lower_case = True):
#     ln = X.shape[0]
#     sentences = []
#     for i in range(ln):
#         tmp = X.iloc[i][f'{text}']
#         tmp_clean = clean_data(tmp,remove_special_char_2lower_case = f"{remove_special_char_2lower_case}")
#         sentences.append(tmp_clean)
#     X["sentences"] = sentences
#     if drop_text == True:
#         X.drop(columns = f'{text}', inplace = True)
#     return X

In [5]:
def clean_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)
   
    # tokenize + remove scpecial characters + set to lower case
    data = text_to_word_sequence(data) 
    
    
    # Remove stopwords
    stop_words = set(stopwords.words('english')) 
    data = [w for w in data if not w in stop_words]         
    
    # Remove digits
    data = ' '.join(word for word in data if not word.isdigit())
    
    
    return text_to_word_sequence(data)

def apply_data_cleaning(X, text, drop_text = False):
    ln = X.shape[0]
    sentences = []
    for i in range(ln):
       # print (f"{i}")
        tmp = X.iloc[i][f'{text}']
        tmp_clean = clean_data(tmp)
        sentences.append(tmp_clean)
    X["sentences"] = sentences
    if drop_text == True:
        X.drop(columns = f'{text}', inplace = True)
    return X

## Help function for Embedding

In [ ]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train_text)
X_test_embed_2 = embedding(word2vec_transfer, X_test_text)

# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

# Data

## Load re-labeled data

In [6]:
with open('../raw_data/DF', 'rb') as handle:
    DF = pickle.load(handle)
DF.dropna(inplace = True)

## Undersample

In [7]:
df = undersample(df= DF, prop = 2)
df.shape
df["label"].value_counts()

0    20182
2    10091
Name: label, dtype: int64

## Oversample

In [8]:
# df = oversample(df =df , prop = 0.3 , seed = 123)
# df.shape
# df["label"].value_counts()

# Data split train, test

In [15]:
# Split in X, y
X = df.copy() 
X.drop(columns = "label", inplace = True)
y = df["label"]

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)
assert(len(X_train)==len(y_train))
assert(len(X_test)==len(y_test))
assert(len(X_train)+len(X_test)==X.shape[0])

# Split X into text and features
X_train_text = X_train[["text"]]
X_test_text = X_test[["text"]]

X_train_features = X_train[["score_punctuation", "score_capital_word"]]
X_test_features = X_test[["score_punctuation", "score_capital_word"]]

# Clean and tokenize text

In [16]:
# print(X_text.head())
X_train_text = apply_data_cleaning(X = X_train_text, text = "text",drop_text = True)#, remove_special_char_2lower_case = True)
X_test_text = apply_data_cleaning(X = X_test_text, text = "text", drop_text = True)#, remove_special_char_2lower_case = True)

X_train_text = list(X_train_text.sentences)
y_train = np.array(y)

X_test_text = list(X_test_text.sentences)
y_test = np.array(y)



# Embedding TF




In [22]:
#print(list(api.info()['models'].keys()))
ll = ['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300', 
 'word2vec-ruscorpora-300', 
 'word2vec-google-news-300', 
 'glove-wiki-gigaword-50', 
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200', 
 'glove-wiki-gigaword-300', 
 'glove-twitter-25', 
 'glove-twitter-50', 
 'glove-twitter-100', 
 'glove-twitter-200', 
 '__testing_word2vec-matrix-synopsis']
word2vec_transfer = api.load("glove-twitter-50")